In [1]:
import re

import pandas as pd
from lightgbm import LGBMClassifier
from sklearn import preprocessing
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("data60.csv", low_memory=False)
df = df.replace(to_replace='?', value=0)
df = df.apply(pd.to_numeric)

In [3]:
df = df.sample(frac=0.001)
df.describe()

,ut_ms,panels [J/m2]@influx,front [J/m2]@influx,back [J/m2]@influx,left [J/m2]@influx,right [J/m2]@influx,up [J/m2]@influx,down [J/m2]@influx,panels-sum1 [J/m2]@influx_sum,panels-sum1-decay0.9 [J/m2]@influx_sum,...,NPWD2792,NPWD2801,NPWD2802,NPWD2821,NPWD2851,NPWD2852,NPWD2871,NPWD2872,NPWD2881,NPWD2882
count,6.600000e+01,6.600000e+01,6.600000e+01,66.0,6.600000e+01,6.600000e+01,6.600000e+01,6.600000e+01,6.000000e+01,6.000000e+01,...,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000
mean,1.348522e+12,1.766890e+09,1.599956e+09,0.0,2.497927e+07,5.102282e+07,7.664980e+07,5.521542e+08,2.020172e+09,1.818155e+09,...,0.001139,0.001539,0.133987,0.005774,0.328104,0.000526,0.000521,0.000744,0.064397,0.003375
std,6.172345e+10,7.621990e+08,7.300135e+08,0.0,1.042876e+08,1.640043e+08,2.116867e+08,4.998137e+08,3.447751e+08,3.102976e+08,...,0.000324,0.000029,0.088989,0.000099,0.147245,0.000010,0.000010,0.000013,0.187848,0.000059
min,1.221109e+12,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.793814e+08,5.214433e+08,...,0.000560,0.001474,0.002240,0.005547,0.005035,0.000501,0.000499,0.000714,0.006457,0.003227
25%,1.302844e+12,1.768255e+09,1.549963e+09,0.0,0.000000e+00,0.000000e+00,0.000000e+00,7.683787e+07,1.803825e+09,1.623442e+09,...,0.000979,0.001524,0.057536,0.005712,0.278083,0.000521,0.000516,0.000737,0.006690,0.003349
50%,1.348790e+12,1.984169e+09,1.760753e+09,0.0,2.889599e+06,0.000000e+00,0.000000e+00,4.138074e+08,2.011179e+09,1.810061e+09,...,0.001213,0.001540,0.134073,0.005783,0.339535,0.000528,0.000523,0.000746,0.006767,0.003388
75%,1.401636e+12,2.229103e+09,2.028441e+09,0.0,1.596690e+07,6.935754e+05,2.377358e+02,9.644152e+08,2.223515e+09,2.001163e+09,...,0.001346,0.001554,0.199803,0.005840,0.402043,0.000533,0.000527,0.000755,0.006844,0.003416
max,1.454548e+12,2.570970e+09,2.569156e+09,0.0,8.357034e+08,8.898539e+08,1.040685e+09,1.928761e+09,2.571126e+09,2.314013e+09,...,0.001779,0.001604,0.310514,0.005991,0.778742,0.000547,0.000543,0.000778,0.720093,0.003491


In [4]:
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [5]:
df.head(5)

,ut_ms,panelsJm2influx,frontJm2influx,backJm2influx,leftJm2influx,rightJm2influx,upJm2influx,downJm2influx,panelssum1Jm2influx_sum,panelssum1decay09Jm2influx_sum,...,NPWD2792,NPWD2801,NPWD2802,NPWD2821,NPWD2851,NPWD2852,NPWD2871,NPWD2872,NPWD2881,NPWD2882
9683,1254222060000,2.120099e+09,1.746184e+09,0.0,3.700275e+06,0.000000e+00,0.0,1.202364e+09,2.120251e+09,1.908226e+09,...,0.000580,0.001492,0.011300,0.005774,0.071356,0.000527,0.000520,0.000744,0.006755,0.003349
33652,1340510460000,1.904914e+09,1.425971e+09,0.0,1.304020e+08,1.503341e+06,0.0,1.227140e+09,1.827937e+09,1.645144e+09,...,0.001064,0.001547,0.053875,0.005744,0.360135,0.000519,0.000524,0.000734,0.720093,0.003394
32212,1335326460000,1.822261e+09,1.618487e+09,0.0,2.042065e+07,4.741922e+05,0.0,8.381684e+08,1.755394e+09,1.579855e+09,...,0.001779,0.001600,0.149136,0.005952,0.286648,0.000542,0.000538,0.000757,0.007041,0.003434
36358,1350252060000,2.346381e+09,2.179111e+09,0.0,1.748435e+07,0.000000e+00,0.0,8.513737e+08,2.322164e+09,2.089948e+09,...,0.001606,0.001567,0.050651,0.005790,0.300159,0.000534,0.000522,0.000744,0.006762,0.003386
46310,1386079260000,1.781429e+09,1.546936e+09,0.0,9.638552e+06,0.000000e+00,0.0,8.833637e+08,1.781456e+09,1.603310e+09,...,0.001350,0.001534,0.140128,0.005824,0.611866,0.000531,0.000524,0.000749,0.702788,0.003491


In [6]:
X = df.iloc[:, 0:484]
y = df.iloc[:, 485:518]
#y = df.iloc[:, -1]

In [7]:
print(X.shape)
print(y.shape)

(66, 484)
(66, 33)


In [15]:
lab = preprocessing.LabelEncoder()
y_trans = lab.fit_transform(y.values.flatten())

In [16]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y_trans, test_size=0.33, random_state=42
)

ValueError: Found input variables with inconsistent numbers of samples: [66, 2178]

In [14]:
lgb = LGBMClassifier(learning_rate=0.09,
                     max_depth=-5,
                     random_state=42)
model_lgb = lgb.fit(X_train, y_train)
y_pred = model_lgb.predict(X_test)
print(classification_report(y_test, y_pred))

NameError: name 'X_train' is not defined